# Importing csv files into postgres database

## Steps

- import csv into pandas dataframe
- clean the table e.g. remove all extra spaces, capital letters, or spaces
- clean headers to match database tables
- import file into db

In [4]:
#Import libraries and modules

import os
import numpy as np
import pandas as pd
import psycopg2
import datetime
import csv
from jproperties import Properties

In [52]:
#Importing csv files into pandas dataframes

Person = pd.read_csv("social_network/person_0_0.csv", sep='|')
PersonEmail = pd.read_csv("social_network/person_email_emailaddress_0_0.csv", sep='|')
PersonLanguage = ""
Organizations = pd.read_csv("social_network/organisation_0_0.csv", sep='|')
PersonStudyAtUniversity = pd.read_csv("social_network/person_studyAt_organisation_0_0.csv", sep='|')
PersonWorkAtCompany = ""
Forum = ""
Post = ""
Message = ""
Comment = ""
Tag = ""
TagClass = ""
PostHasTag = ""
TagHasType = ""
Place = ""
City = ""
Country = ""
Continent = ""
PersonKnowsPerson = ""
ForumHasMember = ""

## Person Table

In [12]:
Person.head()

,id,firstName,lastName,gender,birthday,creationDate,locationIP,browserUsed,place
0,12094627905604,Mirza Kalich,Ali,male,1989-11-02,2012-02-04T06:37:59.956+0000,175.111.0.55,Firefox,794
1,2199023255625,Cheng,Chen,male,1981-12-06,2010-07-12T15:33:44.057+0000,27.98.233.59,Internet Explorer,330
2,2199023255600,Adje van den Berg,Vries,female,1986-08-07,2010-07-14T22:01:21.407+0000,192.101.114.157,Firefox,1253
3,16492674416673,Ali,Ferrer,female,1981-06-30,2012-08-26T07:34:09.342+0000,195.39.218.25,Safari,973
4,2199023255611,Chen,Yang,female,1982-08-28,2010-07-24T07:48:35.041+0000,1.2.2.172,Firefox,427


In [14]:
#replacing headers

"""
CREATE TABLE Person (
    PersonID SERIAL PRIMARY KEY,
    CreationDate TIMESTAMP NOT NULL,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Gender VARCHAR(50),
    Birthday DATE CHECK (Birthday <= CURRENT_DATE),
    BrowserUsed VARCHAR(255),
    LocationIP VARCHAR(255)
);
"""
new_columns = {
    
    "id": "PersonID",
    "creationDate": "CreationDate",
    "firstName": "FirstName",
    "lastName": "LastName",
    "gender": "Gender",
    "birthday": "Birthday",
    "locationIP": "LocationIP",
    "browserUsed": "BrowserUsed",
    "place": "Place" #what is place?
}

Person = Person.rename(columns=new_columns)

Person.head(2)

,PersonID,FirstName,LastName,Gender,Birthday,CreationDate,LocationIP,BrowserUsed,Place
0,12094627905604,Mirza Kalich,Ali,male,1989-11-02,2012-02-04T06:37:59.956+0000,175.111.0.55,Firefox,794
1,2199023255625,Cheng,Chen,male,1981-12-06,2010-07-12T15:33:44.057+0000,27.98.233.59,Internet Explorer,330


In [15]:
#Datatypes
Person.info()


"""
CREATE TABLE Person (
    PersonID BIGINT PRIMARY KEY,
    CreationDate TIMESTAMP NOT NULL,
    FirstName VARCHAR(255),
    LastName VARCHAR(255),
    Gender VARCHAR(50),
    Birthday DATE CHECK (Birthday <= CURRENT_DATE),
    BrowserUsed VARCHAR(255),
    LocationIP VARCHAR(255)
);
"""

# Changes
#   CreationDate to from object to datetime
#   Birthday to datetime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PersonID      88 non-null     int64 
 1   FirstName     88 non-null     object
 2   LastName      88 non-null     object
 3   Gender        88 non-null     object
 4   Birthday      88 non-null     object
 5   CreationDate  88 non-null     object
 6   LocationIP    88 non-null     object
 7   BrowserUsed   88 non-null     object
 8   Place         88 non-null     int64 
dtypes: int64(2), object(7)
memory usage: 6.3+ KB


In [16]:
Person['CreationDate'] = pd.to_datetime(Person['CreationDate'], errors='coerce')


In [18]:
Person['Birthday'] = pd.to_datetime(Person['Birthday'], errors='coerce')


In [19]:
Person.info()

#datatypes ready for import

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   PersonID      88 non-null     int64              
 1   FirstName     88 non-null     object             
 2   LastName      88 non-null     object             
 3   Gender        88 non-null     object             
 4   Birthday      88 non-null     datetime64[ns]     
 5   CreationDate  88 non-null     datetime64[ns, UTC]
 6   LocationIP    88 non-null     object             
 7   BrowserUsed   88 non-null     object             
 8   Place         88 non-null     int64              
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), int64(2), object(5)
memory usage: 6.3+ KB


In [22]:
import psycopg2

try:
    # Establish a connection to the database
    connection = psycopg2.connect(
        dbname="social_network",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

    # Create a cursor object using the connection
    cursor = connection.cursor()

    # Execute a SQL query
    cursor.execute("SELECT * FROM Person")

    # Fetch the results
    rows = cursor.fetchall()

    # Print the results
    for row in rows:
        print(row)

    # Close the cursor and the connection
    cursor.close()
    connection.close()

except psycopg2.Error as e:
    print("Error connecting to PostgreSQL database:", e)


In [26]:
#insert data

query = """INSERT INTO Person (
    PersonID, 
    CreationDate, 
    FirstName, 
    LastName, 
    Gender, 
    Birthday, 
    BrowserUsed, 
    LocationIP) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in Person.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['PersonID']
        CreationDate_value = row['CreationDate']
        FirstName_value = row['FirstName']
        LastName_value = row['LastName']
        Gender_value = row['Gender']
        Birthday_value = row['Birthday']
        BrowserUsed_value = row['BrowserUsed']
        LocationIP_value = row['LocationIP']
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               CreationDate_value,
                               FirstName_value,
                               LastName_value,
                               Gender_value,
                               Birthday_value,
                               BrowserUsed_value,
                               LocationIP_value))
        connection.commit()

# Close the database connection
connection.close()

## PersonEmail

In [29]:
PersonEmail.head()

,Person.id,email
0,12094627905604,Mirza.Kalich12094627905604@gmail.com
1,12094627905604,Mirza.Kalich12094627905604@gmx.com
2,12094627905604,Mirza.Kalich12094627905604@zoho.com
3,2199023255625,Cheng2199023255625@gmail.com
4,2199023255600,Adje.van.den.Berg2199023255600@gmail.com


In [30]:
PersonEmail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Person.id  152 non-null    int64 
 1   email      152 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [33]:
columns = {
    
    "Person.id": "PersonID",
    "email": "Email",
}

PersonEmail = PersonEmail.rename(columns=columns)

PersonEmail.head(2)

,PersonID,Email
0,12094627905604,Mirza.Kalich12094627905604@gmail.com
1,12094627905604,Mirza.Kalich12094627905604@gmx.com


In [36]:
#insert data

query = """INSERT INTO PersonEmail (
    PersonID, 
    Email) 
    VALUES (%s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonEmail.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['PersonID']
        Email_value = row['Email']
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               Email_value))
        connection.commit()

# Close the database connection
connection.close()

## University/Company

In [38]:
Organizations.head()

,id,type,name,url,place
0,0,company,Kam_Air,http://dbpedia.org/resource/Kam_Air,59
1,1,company,Balkh_Airlines,http://dbpedia.org/resource/Balkh_Airlines,59
2,2,company,Khyber_Afghan_Airlines,http://dbpedia.org/resource/Khyber_Afghan_Airl...,59
3,3,company,MarcoPolo_Airways,http://dbpedia.org/resource/MarcoPolo_Airways,59
4,4,company,Pamir_Airways,http://dbpedia.org/resource/Pamir_Airways,59


In [39]:
Organizations['type'].unique()

array(['company', 'university'], dtype=object)

In [ ]:
"""
-- Create University table
CREATE TABLE University (
    UniversityID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL
);

-- Create Company table
CREATE TABLE Company (
    CompanyID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL
);

"""

In [40]:
company_filter = Organizations['type'] == 'company'
university_filter = Organizations['type'] == 'university'

company = Organizations.loc[company_filter]
university = Organizations.loc[university_filter]

In [41]:
company.head()

,id,type,name,url,place
0,0,company,Kam_Air,http://dbpedia.org/resource/Kam_Air,59
1,1,company,Balkh_Airlines,http://dbpedia.org/resource/Balkh_Airlines,59
2,2,company,Khyber_Afghan_Airlines,http://dbpedia.org/resource/Khyber_Afghan_Airl...,59
3,3,company,MarcoPolo_Airways,http://dbpedia.org/resource/MarcoPolo_Airways,59
4,4,company,Pamir_Airways,http://dbpedia.org/resource/Pamir_Airways,59


In [42]:
university.head()

,id,type,name,url,place
1575,1575,university,Paktia_University,http://dbpedia.org/resource/Paktia_University,961
1576,1576,university,Baghlan_University,http://dbpedia.org/resource/Baghlan_University,955
1577,1577,university,Syed_Jamaluddin_Afghan_University,http://dbpedia.org/resource/Syed_Jamaluddin_Af...,958
1578,1578,university,Bost_University,http://dbpedia.org/resource/Bost_University,956
1579,1579,university,Aga_Khan_University,http://dbpedia.org/resource/Aga_Khan_University,775


In [47]:
#insert company data

query = """INSERT INTO Company (
    CompanyID, 
    Name,
    URL) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in company.iterrows():
        # Extract values from the DataFrame
        CompanyID_value = row['id']
        Name_value = row['name']
        URL_value = row['url']
        
        # Execute the SQL query
        cursor.execute(query, (CompanyID_value,
                               Name_value,
                              URL_value))
        connection.commit()

# Close the database connection
connection.close()

In [49]:
#insert university data

query = """INSERT INTO University (
    UniversityID, 
    Name,
    URL) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in university.iterrows():
        # Extract values from the DataFrame
        CompanyID_value = row['id']
        Name_value = row['name']
        URL_value = row['url']
        
        # Execute the SQL query
        cursor.execute(query, (CompanyID_value,
                               Name_value,
                              URL_value))
        connection.commit()

# Close the database connection
connection.close()

In [ ]:
"""
CREATE TABLE PersonStudyAtUniversity (
    PersonID INT NOT NULL,
    UniversityID INT NOT NULL,
    ClassYear INT,
    PRIMARY KEY (PersonID, UniversityID),
    FOREIGN KEY (PersonID) REFERENCES Person(PersonID) ON DELETE CASCADE,
    FOREIGN KEY (UniversityID) REFERENCES University(UniversityID) ON DELETE CASCADE
);
"""

In [53]:
PersonStudyAtUniversity.head()

,Person.id,Organisation.id,classYear
0,2199023255625,2207,2002
1,16492674416673,1596,2002
2,2199023255611,2207,2001
3,13194139533352,1856,2007
4,3298534883391,2833,2005


In [54]:
PersonStudyAtUniversity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Person.id        71 non-null     int64
 1   Organisation.id  71 non-null     int64
 2   classYear        71 non-null     int64
dtypes: int64(3)
memory usage: 1.8 KB


In [56]:
#insert personstudyatuniversity

query = """INSERT INTO PersonStudyAtUniversity (
    PersonID, 
    UniversityID,
    ClassYear) 
    VALUES (%s, %s, %s)"""



# Iterate over rows in the DataFrame and insert data into the table

connection = psycopg2.connect(
        dbname="social_2",
        user="postgres",
        password="password",
        host="localhost",
        port="5432"
    )

   
with connection.cursor() as cursor:
    
    for index, row in PersonStudyAtUniversity.iterrows():
        # Extract values from the DataFrame
        PersonID_value = row['Person.id']
        UniversityID_value = row['Organisation.id']
        ClassYear_value = row['classYear']
        
        
        
        # Execute the SQL query
        cursor.execute(query, (PersonID_value,
                               UniversityID_value,
                              ClassYear_value))
        connection.commit()

# Close the database connection
connection.close()

ProgrammingError: can't adapt type 'numpy.int64'